#### 了解编码问题
常见的编码问题 UnicodeError：  
* UnicodeEncodeError : 把字符串转换成二进制序列时
* UnicodeDecodeError :把二进制序列转换成字符串时
* SyntaxError : 加载 python 模块时有可能抛出

##### 处理UnicodeEncodeError
多数 UTF 编码器只能处理 Unicode 字符的一小部分子集，把文本转换成字节序时，如果目标编码中没有定义某个字符，那就会抛出 UnicodeEncodeError 异常

In [1]:
city = 'Sāo Paulo'
city.encode('utf-8')

b'S\xc4\x81o Paulo'

In [2]:
# utf 编码可以处理任何字符串
city.encode('utf-16')

b'\xff\xfeS\x00\x01\x01o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [4]:
# 'iso8859_1' 无法处理 'ā' 字符
city.encode('iso8859_1')

UnicodeEncodeError: 'latin-1' codec can't encode character '\u0101' in position 1: ordinal not in range(256)

In [5]:
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\u0101' in position 1: character maps to <undefined>

In [6]:
# errors='ignore' 可以跳过无法编码的字符
city.encode('cp437', errors='ignore')

b'So Paulo'

In [7]:
# errors='replace' 把无法编码的字符替换成 '?'; 数据损坏了，但用户知道出了问题
city.encode('cp437', errors='replace')

b'S?o Paulo'

In [8]:
# errors='xmlcharrefreplace' 把无法编码的字符替换成 XML 实体
city.encode('cp437', errors='xmlcharrefreplace')

b'S&#257;o Paulo'

编解码器的处理方式是可扩展的。你可以为 error 参数注册额外的字符串，方法是把一个名称和一个错误处理函数传给 codecs.regsiter_error 函数

#### 处理 UnicodeDecodeError
不是每一个字节都包含有效的 ASCII 字符，也不是每个字符序列都是有效的 UTF-8 或 UTF-16。
但很多陈旧的 8 位编码 —— 如 'cp1252'、'iso8859_1' 和 'koi8_r' —— 能解码任何字节流而不抛出错误，但有可能会出现乱码。

In [9]:
# 字节序列解码成字符串
octets = b'Montr\xe9al'
octets.decode('cp1252')

'Montréal'

In [10]:
octets.decode('iso8859_7')

'Montrιal'

In [11]:
octets.decode('koi8_r')

'MontrИal'

In [12]:
octets.decode('utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [13]:
octets.decode('utf-8', errors= 'replace')

'Montr�al'

#### 使用预期之外的编码加载模块抛出 SyntaxError
Python 3 默认使用 UTF-8 编码，如果 .py 模块中包含 UTF-8 之外的数据，且没有声明编码，则会抛出 SyntaxError
使用 coding 注释来声明编码
```python
> # coding: cp1252
```

##### 判断字符编码
统一字符编码侦测包 Chardet